In [1]:

import time
import re
import os
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [12]:
country_dict = {
    ## 1_10
    # "United States of America": "US",
    # "Australia": "AU",
    # "Canada": "CA",
    # "China": "CN",
    # "France": "FR",
    # "Germany": "DE",
    # "United Kingdom": "GB",
    # "Italy": "IT",
    # "Japan": "JP",
    # "South Korea": "KR",

    ## 2_10
    # "Russia": "RU",
    # "Algeria": "DZ",
    # "Angola": "AO",
    # "Argentina": "AR",
    # "Austria": "AT",
    # "Azerbaijan": "AZ",
    # "Bahrain": "BH",
    # "Barbados": "BB",
    # "Belarus": "BY",
    # "Belgium": "BE",

    ## 3_10
    "Bermuda": "BM",
    "Brazil": "BR",
    "Bulgaria": "BG",
    "Chile": "CL",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Croatia": "HR",
    "Cyprus": "CY",
    "Czech Republic": "CZ",
    "Denmark": "DK",

    ## 4_10
    # "Dominican Republic": "DO",
    # "Ecuador": "EC",
    # "Egypt": "EG",
    # "El Salvador": "SV",
    # "Finland": "FI",
    # "Ghana": "GH",
    # "Greece": "GR",
    # "Guatemala": "GT",
    # "Hong Kong": "HK",
    # "Hungary": "HU",

    ## 5_10
    # "India": "IN",
    # "Indonesia": "ID",
    # "Ireland": "IE",
    # "Israel": "IL",
    # "Kazakhstan": "KZ",
    # "Kenya": "KE",
    # "Kuwait": "KW",
    # "Lebanon": "LB",
    # "Lithuania": "LT",
    # "Luxembourg": "LU",

    ## 6_10
    # "Macau": "MO",
    # "Madagascar": "MG",
    # "Malaysia": "MY",
    # "Malta": "MT",
    # "Mexico": "MX",
    # "Netherlands": "NL",
    # "New Zealand": "NZ",
    # "Nigeria": "NG",
    # "Norway": "NO",
    # "Oman": "OM",

    ## 7_10
    # "Pakistan": "PK",
    # "Panama": "PA",
    # "Peru": "PE",
    # "Philippines": "PH",
    # "Poland": "PL",
    # "Portugal": "PT",
    # "Qatar": "QA",
    # "Romania": "RO",
    # "Saudi Arabia": "SA",
    # "Serbia": "RS",

    ## 8_10
    # "Singapore": "SG",
    # "Slovakia": "SK",
    # "Slovenia": "SI",
    # "South Africa": "ZA",
    # "Spain": "ES",
    # "Sri Lanka": "LK",
    # "Sweden": "SE",
    # "Switzerland": "CH",
    # "Taiwan": "TW",
    # "Thailand": "TH",

    ## 9_10
    # "Tunisia": "TN",
    # "Turkey": "TR",
    # "Ukraine": "UA",
    # "United Arab Emirates": "AE",
    # "Uruguay": "UY",
    # "Uzbekistan": "UZ",
    # "Venezuela": "VE",
    # "Vietnam": "VN",
    # "Bolivia": "BO",
    # "Cambodia": "KH",

    ## 10_12
    # "Estonia": "EE",
    # "Latvia": "LV",
    # "Nicaragua": "NI",
    # "Paraguay": "PY",
    # "Afghanistan": "AF",
    # "Georgia": "GE",
    # "Iraq": "IQ",
    # "Libya": "LY",
    # "Morocco": "MA",
    # "Mozambique": "MZ",
    # "Myanmar": "MM",
    # "Yemen": "YE"
}


dates = ['2024-12-30', '2024-12-31']

categories = {
    'Entertainment': '6016',
    'Education': '6017'
}

base_url = 'https://app.sensortower.com/top-charts?os=ios&device=iphone&'
columns = ["Rank", "Country", "Name", "Publisher", "Category", "Rating", "Price", "Rating_count", "Date"]
neumeric_columns = ["Rank", "Rating", "Price", "Number_of_Ratings"]


In [3]:
def extract_price_value(price_string):
    # Regular expression to match integers or floats
    match = re.search(r'\b\d+(\.\d+)?\b', price_string)
    if match:
        return float(match.group())  # Convert to float
    return 0

def extract_rating_value(rating_string):
    match = re.search(r"(\d+\.\d+)", rating_string)
    if match:
        return float(match.group(1))
    return 0

In [8]:
def extract_data_for_country(country, category, date):
    url = base_url + f'country={country_dict[country]}&category={categories[category]}&date={date}'
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    time.sleep(3)
    rows = driver.find_element(By.TAG_NAME, 'table').find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
    records = []
    for idx, row in enumerate(rows):
        data = row.find_elements(By.TAG_NAME, 'td')
        paid_app_data = data[2].find_element(By.TAG_NAME, 'span').find_element(By.TAG_NAME, 'div').find_element(By.TAG_NAME, 'div')
        data = paid_app_data.text.split('\n')

        rating_data = paid_app_data.find_elements(By.TAG_NAME, 'div')[4].find_element(By.TAG_NAME, 'span').get_attribute('aria-label')
        ls = [idx + 1, country, data[0], data[1], category, extract_rating_value(rating_string=rating_data), extract_price_value(data[4]), int(data[2].strip("()").replace(",", "")), date]
        record = dict(zip(columns, ls))
        records.append(record)
    driver.quit()
    return records

In [9]:
data = []
for country in country_dict.keys():
    for category in categories.keys():
        for date in dates:
            data.extend(extract_data_for_country(country=country, category=category, date=date))

In [ ]:
print(len(data))

500
1000


In [15]:
df = pd.DataFrame(data)
df.head()


,Rank,Country,Name,Publisher,Category,Rating,Price,Rating_count,Date
0,1,Bermuda,Youtify + for Spotify Premium,"KISSAPP, S.L.",Entertainment,4.33,1.99,3,2024-12-30
1,2,Bermuda,Pocket God,Bolt Creative,Entertainment,0.00,0.99,0,2024-12-30
2,3,Bermuda,Drinking games 3000+ questions,Charlie Sendrowski,Entertainment,0.00,0.99,0,2024-12-30
3,4,Bermuda,MuvaMoji by Amber Rose,"AppMoji, Inc.",Entertainment,5.00,1.99,1,2024-12-30
4,5,Bermuda,737 Flight Simulator,Justyna Zablocka,Entertainment,0.00,1.99,0,2024-12-30


In [16]:
directory = "data"
file_name = "3_10.csv"
file_path = os.path.join(directory, file_name)

os.makedirs(directory, exist_ok=True)

df.to_csv(file_path, index=True)